In [1]:
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # specify the path of the folder containing "file_name" :
    path_to_file = '/content/gdrive/My Drive/Big Data/'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/Big Data/
/content/gdrive/My Drive/Big Data


In [2]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('tweet_retweet_network.csv', dtype={'user_id':str})
df

,tweet_id,user_id_retweet_df,user_id_tweet_df,news_id,is_fake_news
0,1033706162695356417,7.873112e+08,4.335085e+07,politifact99,0.0
1,1033706162695356417,7.873112e+08,4.335085e+07,politifact340,0.0
2,1035580865160638464,3.338247e+09,1.629771e+07,politifact99,0.0
3,1035580865160638464,3.338247e+09,1.629771e+07,politifact340,0.0
4,934206237708865537,7.543102e+17,5.820642e+06,politifact99,0.0
...,...,...,...,...,...
573632,813192381583466496,NaN,3.001448e+09,politifact13600,1.0
573633,812814063763918848,NaN,5.256945e+08,politifact13600,1.0
573634,918510122363744257,NaN,1.525687e+09,politifact14621,1.0
573635,917522953390223366,NaN,7.889922e+17,politifact14621,1.0


In [7]:
df.isna().sum()

tweet_id                   0
user_id_retweet_df    484610
user_id_tweet_df       29610
news_id                    1
is_fake_news               1
dtype: int64

In [8]:
df = df[df['user_id_tweet_df'].notna()]
df

,tweet_id,user_id_retweet_df,user_id_tweet_df,news_id,is_fake_news
0,1033706162695356417,7.873112e+08,4.335085e+07,politifact99,0.0
1,1033706162695356417,7.873112e+08,4.335085e+07,politifact340,0.0
2,1035580865160638464,3.338247e+09,1.629771e+07,politifact99,0.0
3,1035580865160638464,3.338247e+09,1.629771e+07,politifact340,0.0
4,934206237708865537,7.543102e+17,5.820642e+06,politifact99,0.0
...,...,...,...,...,...
573632,813192381583466496,NaN,3.001448e+09,politifact13600,1.0
573633,812814063763918848,NaN,5.256945e+08,politifact13600,1.0
573634,918510122363744257,NaN,1.525687e+09,politifact14621,1.0
573635,917522953390223366,NaN,7.889922e+17,politifact14621,1.0


In [9]:
df_followers = pd.read_csv('tweet_retweet_follower_network.csv', dtype={'user_id':str})
df_followers

,followers,user_id,is_fake_news,follow_or_retweet
0,7.873112e+08,43350851,0.0,retweet
1,3.338247e+09,16297707,0.0,retweet
2,7.543102e+17,5820642,0.0,retweet
3,2.157919e+09,1716121,0.0,retweet
4,4.867759e+09,31056977,0.0,retweet
...,...,...,...,...
4032402,3.306284e+09,3416678236,NaN,follower
4032403,4.102376e+09,3416678236,NaN,follower
4032404,4.165642e+09,3416678236,NaN,follower
4032405,3.847086e+08,3416678236,NaN,follower


In [11]:
df_followers['is_fake_news'].unique().tolist()

[0.0, 1.0, nan]

In [12]:
df_followers.isna().sum()

followers             267908
user_id                 2174
is_fake_news         3683264
follow_or_retweet          0
dtype: int64

In [14]:
!pip install pyspark
!pip install graphframes
!apt-get install scala
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 66.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=0701ea0d4dd0673f3e418c9d8bc3230666f318101951433078f6d3fa7f18a9cc
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 154 kB 39.7 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libhawtjni-runtime-java libjansi-java libjansi-native-java libjline2-java
  scala-library scala-parser-combinators scala-xml
Suggested packages:
  scala-doc
The following NEW packages will be installed:
  libhawtjni-runtime-java libjansi-java libjansi-native-java libjline2-java
  scala scala-library scala-parser-combinators scala-xml
0 upgraded, 8 newly installed, 0 to remove an

In [38]:
#import packages
import os
import sys

from pyspark import SparkConf, SparkContext
import os
import math
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType 
from graphframes import *

import matplotlib.pyplot as plt
%matplotlib inline

import math

In [16]:
spark = SparkSession.builder.master("local[*]").config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12").getOrCreate()

In [20]:
df = spark.read.csv('tweet_retweet_network.csv', header = True)
df.show()

+-------------------+-------------------+----------------+-------------+------------+
|           tweet_id| user_id_retweet_df|user_id_tweet_df|      news_id|is_fake_news|
+-------------------+-------------------+----------------+-------------+------------+
|1033706162695356417|          787311228|        43350851| politifact99|         0.0|
|1033706162695356417|          787311228|        43350851|politifact340|         0.0|
|1035580865160638464|         3338246572|        16297707| politifact99|         0.0|
|1035580865160638464|         3338246572|        16297707|politifact340|         0.0|
| 934206237708865537| 754310205546954757|         5820642| politifact99|         0.0|
|1035367234569269248|         2157919340|         1716121| politifact99|         0.0|
|1035367234569269248|         2157919340|         1716121|politifact340|         0.0|
|1034217862313861120|         4867759271|        31056977| politifact99|         0.0|
|1034217862313861120|         4867759271|        31056

In [23]:
tweets_df = df.select('user_id_tweet_df','news_id', 'is_fake_news')
tweets_df.show()

+----------------+-------------+------------+
|user_id_tweet_df|      news_id|is_fake_news|
+----------------+-------------+------------+
|        43350851| politifact99|         0.0|
|        43350851|politifact340|         0.0|
|        16297707| politifact99|         0.0|
|        16297707|politifact340|         0.0|
|         5820642| politifact99|         0.0|
|         1716121| politifact99|         0.0|
|         1716121|politifact340|         0.0|
|        31056977| politifact99|         0.0|
|        31056977|politifact340|         0.0|
|        13115682| politifact99|         0.0|
|        13115682|politifact340|         0.0|
|        13115682|politifact542|         0.0|
|        22696550| politifact99|         0.0|
|        22696550|politifact340|         0.0|
|       135247359| politifact99|         0.0|
|       135247359|politifact340|         0.0|
|        16312576| politifact99|         0.0|
|        16312576|politifact340|         0.0|
|        17112878| politifact99|  

In [47]:
tweets_df_fake = tweets_df.filter('is_fake_news = 1.0')
tweets_df_fake.show()

+-------------------+---------------+------------+
|   user_id_tweet_df|        news_id|is_fake_news|
+-------------------+---------------+------------+
|               null|politifact13905|         1.0|
|           39744993|politifact14667|         1.0|
|           82770649|politifact14667|         1.0|
|           21474769|politifact14667|         1.0|
| 824797212425191425|politifact14667|         1.0|
|          110918004|politifact14667|         1.0|
|          184929716|politifact14667|         1.0|
|          110918004|politifact14667|         1.0|
|          191589342|politifact14667|         1.0|
|          110918004|politifact14667|         1.0|
|         4520399652|politifact14667|         1.0|
|          110918004|politifact14667|         1.0|
| 866690638909104129|politifact14667|         1.0|
|         2850251396|politifact14548|         1.0|
|           62088756|politifact14548|         1.0|
| 746236470684311552|politifact14548|         1.0|
|           47299616|politifact

In [48]:
tweets_df_real = tweets_df.filter('is_fake_news = 0.0')
tweets_df_real.show()

+----------------+-------------+------------+
|user_id_tweet_df|      news_id|is_fake_news|
+----------------+-------------+------------+
|        43350851| politifact99|         0.0|
|        43350851|politifact340|         0.0|
|        16297707| politifact99|         0.0|
|        16297707|politifact340|         0.0|
|         5820642| politifact99|         0.0|
|         1716121| politifact99|         0.0|
|         1716121|politifact340|         0.0|
|        31056977| politifact99|         0.0|
|        31056977|politifact340|         0.0|
|        13115682| politifact99|         0.0|
|        13115682|politifact340|         0.0|
|        13115682|politifact542|         0.0|
|        22696550| politifact99|         0.0|
|        22696550|politifact340|         0.0|
|       135247359| politifact99|         0.0|
|       135247359|politifact340|         0.0|
|        16312576| politifact99|         0.0|
|        16312576|politifact340|         0.0|
|        17112878| politifact99|  

In [49]:
tweets_df_fake = tweets_df_fake.na.drop()
tweets_df_fake.show()

+-------------------+---------------+------------+
|   user_id_tweet_df|        news_id|is_fake_news|
+-------------------+---------------+------------+
|           39744993|politifact14667|         1.0|
|           82770649|politifact14667|         1.0|
|           21474769|politifact14667|         1.0|
| 824797212425191425|politifact14667|         1.0|
|          110918004|politifact14667|         1.0|
|          184929716|politifact14667|         1.0|
|          110918004|politifact14667|         1.0|
|          191589342|politifact14667|         1.0|
|          110918004|politifact14667|         1.0|
|         4520399652|politifact14667|         1.0|
|          110918004|politifact14667|         1.0|
| 866690638909104129|politifact14667|         1.0|
|         2850251396|politifact14548|         1.0|
|           62088756|politifact14548|         1.0|
| 746236470684311552|politifact14548|         1.0|
|           47299616|politifact14548|         1.0|
|1004813571161755648|politifact

In [50]:
tweets_df_real = tweets_df_real.na.drop()
tweets_df_real.show()

+----------------+-------------+------------+
|user_id_tweet_df|      news_id|is_fake_news|
+----------------+-------------+------------+
|        43350851| politifact99|         0.0|
|        43350851|politifact340|         0.0|
|        16297707| politifact99|         0.0|
|        16297707|politifact340|         0.0|
|         5820642| politifact99|         0.0|
|         1716121| politifact99|         0.0|
|         1716121|politifact340|         0.0|
|        31056977| politifact99|         0.0|
|        31056977|politifact340|         0.0|
|        13115682| politifact99|         0.0|
|        13115682|politifact340|         0.0|
|        13115682|politifact542|         0.0|
|        22696550| politifact99|         0.0|
|        22696550|politifact340|         0.0|
|       135247359| politifact99|         0.0|
|       135247359|politifact340|         0.0|
|        16312576| politifact99|         0.0|
|        16312576|politifact340|         0.0|
|        17112878| politifact99|  

In [51]:
fake_tweets_rdd = tweets_df_fake.rdd.map(tuple)
fake_tweets_rdd.take(10)

[('39744993', 'politifact14667', '1.0'),
 ('82770649', 'politifact14667', '1.0'),
 ('21474769', 'politifact14667', '1.0'),
 ('824797212425191425', 'politifact14667', '1.0'),
 ('110918004', 'politifact14667', '1.0'),
 ('184929716', 'politifact14667', '1.0'),
 ('110918004', 'politifact14667', '1.0'),
 ('191589342', 'politifact14667', '1.0'),
 ('110918004', 'politifact14667', '1.0'),
 ('4520399652', 'politifact14667', '1.0')]

In [54]:
fake_tweets_rdd.take(1)[0][2]

'1.0'

In [55]:
fake_tweets_map = fake_tweets_rdd.map(lambda x: (x[0], 1))
fake_tweets_map.take(10)

[('39744993', 1),
 ('82770649', 1),
 ('21474769', 1),
 ('824797212425191425', 1),
 ('110918004', 1),
 ('184929716', 1),
 ('110918004', 1),
 ('191589342', 1),
 ('110918004', 1),
 ('4520399652', 1)]

In [56]:
fake_tweets_count = fake_tweets_map.reduceByKey(lambda n1,n2: n1+n2)
fake_tweets_count.take(10)

[('21474769', 4),
 ('824797212425191425', 1),
 ('110918004', 25),
 ('191589342', 1),
 ('62088756', 1),
 ('3097431114', 6),
 ('2862646276', 1),
 ('835117966165364736', 3),
 ('2905658255', 2),
 ('935085971451924480', 2)]

In [58]:
real_tweets_rdd = tweets_df_real.rdd.map(tuple)
real_tweets_rdd.take(10)

[('43350851', 'politifact99', '0.0'),
 ('43350851', 'politifact340', '0.0'),
 ('16297707', 'politifact99', '0.0'),
 ('16297707', 'politifact340', '0.0'),
 ('5820642', 'politifact99', '0.0'),
 ('1716121', 'politifact99', '0.0'),
 ('1716121', 'politifact340', '0.0'),
 ('31056977', 'politifact99', '0.0'),
 ('31056977', 'politifact340', '0.0'),
 ('13115682', 'politifact99', '0.0')]

In [59]:
real_tweets_map = real_tweets_rdd.map(lambda x: (x[0], 1))
real_tweets_map.take(10)

[('43350851', 1),
 ('43350851', 1),
 ('16297707', 1),
 ('16297707', 1),
 ('5820642', 1),
 ('1716121', 1),
 ('1716121', 1),
 ('31056977', 1),
 ('31056977', 1),
 ('13115682', 1)]

In [60]:
real_tweets_count = real_tweets_map.reduceByKey(lambda n1,n2: n1+n2)
real_tweets_count.take(10)

[('1716121', 7),
 ('13115682', 20),
 ('135247359', 4),
 ('17112878', 426),
 ('993451813', 1),
 ('50174697', 2),
 ('17921386', 2),
 ('18812572', 11),
 ('2463977196', 1),
 ('994358258646355968', 2)]

In [65]:
all_tweets_count = real_tweets_count.fullOuterJoin(fake_tweets_count)
all_tweets_count = all_tweets_count.map(lambda x: (x[0], x[1][0], x[1][1]))
all_tweets_count.take(10)

[('1716121', 7, None),
 ('13115682', 20, 1),
 ('135247359', 4, 2),
 ('17112878', 426, None),
 ('993451813', 1, 3),
 ('50174697', 2, 1),
 ('17921386', 2, None),
 ('18812572', 11, None),
 ('2463977196', 1, None),
 ('994358258646355968', 2, None)]

In [91]:
rdd_columns = ['user_id', 'tweet_real_news_count', 'tweet_fake_news_count']
tweet_count_df = spark.createDataFrame(all_tweets_count, schema = rdd_columns)
tweet_count_df = tweet_count_df.na.fill(0)
tweet_count_df.show(10)

+------------------+---------------------+---------------------+
|           user_id|tweet_real_news_count|tweet_fake_news_count|
+------------------+---------------------+---------------------+
|           1716121|                    7|                    0|
|          13115682|                   20|                    1|
|         135247359|                    4|                    2|
|          17112878|                  426|                    0|
|         993451813|                    1|                    3|
|          50174697|                    2|                    1|
|          17921386|                    2|                    0|
|          18812572|                   11|                    0|
|        2463977196|                    1|                    0|
|994358258646355968|                    2|                    0|
+------------------+---------------------+---------------------+
only showing top 10 rows



In [92]:
retweets_df = df.select('user_id_retweet_df','news_id', 'is_fake_news')
retweets_df.show()

+-------------------+-------------+------------+
| user_id_retweet_df|      news_id|is_fake_news|
+-------------------+-------------+------------+
|          787311228| politifact99|         0.0|
|          787311228|politifact340|         0.0|
|         3338246572| politifact99|         0.0|
|         3338246572|politifact340|         0.0|
| 754310205546954757| politifact99|         0.0|
|         2157919340| politifact99|         0.0|
|         2157919340|politifact340|         0.0|
|         4867759271| politifact99|         0.0|
|         4867759271|politifact340|         0.0|
|          370794560| politifact99|         0.0|
|          370794560|politifact340|         0.0|
|          370794560|politifact542|         0.0|
|1034479942895951873| politifact99|         0.0|
|1034479942895951873|politifact340|         0.0|
|          254708177| politifact99|         0.0|
|          254708177|politifact340|         0.0|
|           27128191| politifact99|         0.0|
|           27128191

In [93]:
retweets_df = retweets_df.na.drop()
retweets_df.show()

+-------------------+-------------+------------+
| user_id_retweet_df|      news_id|is_fake_news|
+-------------------+-------------+------------+
|          787311228| politifact99|         0.0|
|          787311228|politifact340|         0.0|
|         3338246572| politifact99|         0.0|
|         3338246572|politifact340|         0.0|
| 754310205546954757| politifact99|         0.0|
|         2157919340| politifact99|         0.0|
|         2157919340|politifact340|         0.0|
|         4867759271| politifact99|         0.0|
|         4867759271|politifact340|         0.0|
|          370794560| politifact99|         0.0|
|          370794560|politifact340|         0.0|
|          370794560|politifact542|         0.0|
|1034479942895951873| politifact99|         0.0|
|1034479942895951873|politifact340|         0.0|
|          254708177| politifact99|         0.0|
|          254708177|politifact340|         0.0|
|           27128191| politifact99|         0.0|
|           27128191

In [94]:
retweets_df_fake = retweets_df.filter('is_fake_news = 1.0')
retweets_df_fake.show()

+-------------------+---------------+------------+
| user_id_retweet_df|        news_id|is_fake_news|
+-------------------+---------------+------------+
|         3162969997|politifact13905|         1.0|
| 885692002591006720|politifact14667|         1.0|
|         1917218150|politifact14667|         1.0|
|          255928301|politifact14667|         1.0|
| 822271632383295493|politifact14667|         1.0|
|           17597022|politifact14667|         1.0|
|           22824243|politifact14667|         1.0|
|         1378552322|politifact14667|         1.0|
|          188343111|politifact14667|         1.0|
|1017951685224759296|politifact14667|         1.0|
|            9924742|politifact14667|         1.0|
|          915331472|politifact14667|         1.0|
| 960571982780731392|politifact14667|         1.0|
|         2716850508|politifact14548|         1.0|
| 948025288356642816|politifact14548|         1.0|
| 759421234375331840|politifact14548|         1.0|
|          103253996|politifact

In [95]:
retweets_df_real = retweets_df.filter('is_fake_news = 0.0')
retweets_df_real.show()

+-------------------+-------------+------------+
| user_id_retweet_df|      news_id|is_fake_news|
+-------------------+-------------+------------+
|          787311228| politifact99|         0.0|
|          787311228|politifact340|         0.0|
|         3338246572| politifact99|         0.0|
|         3338246572|politifact340|         0.0|
| 754310205546954757| politifact99|         0.0|
|         2157919340| politifact99|         0.0|
|         2157919340|politifact340|         0.0|
|         4867759271| politifact99|         0.0|
|         4867759271|politifact340|         0.0|
|          370794560| politifact99|         0.0|
|          370794560|politifact340|         0.0|
|          370794560|politifact542|         0.0|
|1034479942895951873| politifact99|         0.0|
|1034479942895951873|politifact340|         0.0|
|          254708177| politifact99|         0.0|
|          254708177|politifact340|         0.0|
|           27128191| politifact99|         0.0|
|           27128191

In [96]:
fake_retweets_rdd = retweets_df_fake.rdd.map(tuple)
fake_retweets_rdd.take(10)

[('3162969997', 'politifact13905', '1.0'),
 ('885692002591006720', 'politifact14667', '1.0'),
 ('1917218150', 'politifact14667', '1.0'),
 ('255928301', 'politifact14667', '1.0'),
 ('822271632383295493', 'politifact14667', '1.0'),
 ('17597022', 'politifact14667', '1.0'),
 ('22824243', 'politifact14667', '1.0'),
 ('1378552322', 'politifact14667', '1.0'),
 ('188343111', 'politifact14667', '1.0'),
 ('1017951685224759296', 'politifact14667', '1.0')]

In [97]:
real_retweets_rdd = retweets_df_real.rdd.map(tuple)
real_retweets_rdd.take(10)

[('787311228', 'politifact99', '0.0'),
 ('787311228', 'politifact340', '0.0'),
 ('3338246572', 'politifact99', '0.0'),
 ('3338246572', 'politifact340', '0.0'),
 ('754310205546954757', 'politifact99', '0.0'),
 ('2157919340', 'politifact99', '0.0'),
 ('2157919340', 'politifact340', '0.0'),
 ('4867759271', 'politifact99', '0.0'),
 ('4867759271', 'politifact340', '0.0'),
 ('370794560', 'politifact99', '0.0')]

In [98]:
fake_retweets_map = fake_retweets_rdd.map(lambda x: (x[0], 1))
fake_retweets_map.take(10)

[('3162969997', 1),
 ('885692002591006720', 1),
 ('1917218150', 1),
 ('255928301', 1),
 ('822271632383295493', 1),
 ('17597022', 1),
 ('22824243', 1),
 ('1378552322', 1),
 ('188343111', 1),
 ('1017951685224759296', 1)]

In [99]:
real_retweets_map = real_retweets_rdd.map(lambda x: (x[0], 1))
real_retweets_map.take(10)

[('787311228', 1),
 ('787311228', 1),
 ('3338246572', 1),
 ('3338246572', 1),
 ('754310205546954757', 1),
 ('2157919340', 1),
 ('2157919340', 1),
 ('4867759271', 1),
 ('4867759271', 1),
 ('370794560', 1)]

In [100]:
fake_retweets_count = fake_retweets_map.reduceByKey(lambda n1,n2: n1+n2)
fake_retweets_count.take(10)

[('3162969997', 1),
 ('1917218150', 1),
 ('255928301', 1),
 ('822271632383295493', 1),
 ('17597022', 1),
 ('22824243', 1),
 ('1378552322', 1),
 ('1017951685224759296', 2),
 ('9924742', 1),
 ('960571982780731392', 1)]

In [101]:
real_retweets_count = real_retweets_map.reduceByKey(lambda n1,n2: n1+n2)
real_retweets_count.take(10)

[('3338246572', 2),
 ('2157919340', 2),
 ('4867759271', 2),
 ('370794560', 3),
 ('1325337348', 2),
 ('1924285831', 1),
 ('2548750220', 1),
 ('3107597132', 1),
 ('18907297', 1),
 ('479832877', 2)]

In [102]:
all_retweets_count = real_retweets_count.fullOuterJoin(fake_retweets_count)
all_retweets_count = all_retweets_count.map(lambda x: (x[0], x[1][0], x[1][1]))
all_retweets_count.take(10)

[('3338246572', 2, None),
 ('2157919340', 2, 1),
 ('4867759271', 2, None),
 ('370794560', 3, None),
 ('1325337348', 2, 2),
 ('1924285831', 1, None),
 ('2548750220', 1, None),
 ('3107597132', 1, 1),
 ('18907297', 1, None),
 ('479832877', 2, None)]

In [103]:
rdd_columns_2 = ['user_id', 'retweet_real_news_count', 'retweet_fake_news_count']
retweet_count_df = spark.createDataFrame(all_retweets_count, schema = rdd_columns_2)
retweet_count_df = retweet_count_df.na.fill(0)
retweet_count_df.show(10)

+----------+-----------------------+-----------------------+
|   user_id|retweet_real_news_count|retweet_fake_news_count|
+----------+-----------------------+-----------------------+
|3338246572|                      2|                      0|
|2157919340|                      2|                      1|
|4867759271|                      2|                      0|
| 370794560|                      3|                      0|
|1325337348|                      2|                      2|
|1924285831|                      1|                      0|
|2548750220|                      1|                      0|
|3107597132|                      1|                      1|
|  18907297|                      1|                      0|
| 479832877|                      2|                      0|
+----------+-----------------------+-----------------------+
only showing top 10 rows



In [104]:
tweet_count_df = tweet_count_df.withColumn("tweet_count", tweet_count_df["tweet_real_news_count"] + tweet_count_df["tweet_fake_news_count"])
tweet_count_df.show(10)

+------------------+---------------------+---------------------+-----------+
|           user_id|tweet_real_news_count|tweet_fake_news_count|tweet_count|
+------------------+---------------------+---------------------+-----------+
|           1716121|                    7|                    0|          7|
|          13115682|                   20|                    1|         21|
|         135247359|                    4|                    2|          6|
|          17112878|                  426|                    0|        426|
|         993451813|                    1|                    3|          4|
|          50174697|                    2|                    1|          3|
|          17921386|                    2|                    0|          2|
|          18812572|                   11|                    0|         11|
|        2463977196|                    1|                    0|          1|
|994358258646355968|                    2|                    0|          2|

In [105]:
retweet_count_df = retweet_count_df.withColumn("retweet_count", retweet_count_df["retweet_real_news_count"] + retweet_count_df["retweet_fake_news_count"])
retweet_count_df.show(10)

+----------+-----------------------+-----------------------+-------------+
|   user_id|retweet_real_news_count|retweet_fake_news_count|retweet_count|
+----------+-----------------------+-----------------------+-------------+
|3338246572|                      2|                      0|            2|
|2157919340|                      2|                      1|            3|
|4867759271|                      2|                      0|            2|
| 370794560|                      3|                      0|            3|
|1325337348|                      2|                      2|            4|
|1924285831|                      1|                      0|            1|
|2548750220|                      1|                      0|            1|
|3107597132|                      1|                      1|            2|
|  18907297|                      1|                      0|            1|
| 479832877|                      2|                      0|            2|
+----------+-------------

In [111]:
tweet_retweet_counts_df = tweet_count_df.join(retweet_count_df,'user_id',"outer")
tweet_retweet_counts_df = tweet_retweet_counts_df.na.fill(0)
tweet_retweet_counts_df.show(10)

+-------------------+---------------------+---------------------+-----------+-----------------------+-----------------------+-------------+
|            user_id|tweet_real_news_count|tweet_fake_news_count|tweet_count|retweet_real_news_count|retweet_fake_news_count|retweet_count|
+-------------------+---------------------+---------------------+-----------+-----------------------+-----------------------+-------------+
|1000011520099737600|                    1|                    0|          1|                      0|                      0|            0|
|1000015310651645953|                    1|                    0|          1|                      0|                      0|            0|
|1000016148920913920|                    1|                    0|          1|                      0|                      0|            0|
|1000017210381819904|                    1|                    0|          1|                      0|                      0|            0|
|          100002112

In [112]:
tweet_retweet_counts_df = tweet_retweet_counts_df.withColumn("tweet_retweet_fake_count", tweet_retweet_counts_df["tweet_fake_news_count"] + tweet_retweet_counts_df["retweet_fake_news_count"])
tweet_retweet_counts_df = tweet_retweet_counts_df.withColumn("tweet_retweet_real_count", tweet_retweet_counts_df["tweet_real_news_count"] + tweet_retweet_counts_df["retweet_real_news_count"])
tweet_retweet_counts_df = tweet_retweet_counts_df.withColumn("tweet_retweet_count", tweet_retweet_counts_df["retweet_count"] + tweet_retweet_counts_df["tweet_count"])
tweet_retweet_counts_df.show(10)

+-------------------+---------------------+---------------------+-----------+-----------------------+-----------------------+-------------+------------------------+------------------------+-------------------+
|            user_id|tweet_real_news_count|tweet_fake_news_count|tweet_count|retweet_real_news_count|retweet_fake_news_count|retweet_count|tweet_retweet_fake_count|tweet_retweet_real_count|tweet_retweet_count|
+-------------------+---------------------+---------------------+-----------+-----------------------+-----------------------+-------------+------------------------+------------------------+-------------------+
|1000011520099737600|                    1|                    0|          1|                      0|                      0|            0|                       0|                       1|                  1|
|1000015310651645953|                    1|                    0|          1|                      0|                      0|            0|                     

In [113]:
tweet_retweet_counts_df = tweet_retweet_counts_df.withColumn("percentage_fake", tweet_retweet_counts_df["tweet_retweet_fake_count"] / tweet_retweet_counts_df["tweet_retweet_count"])
tweet_retweet_counts_df.show(10)

+-------------------+---------------------+---------------------+-----------+-----------------------+-----------------------+-------------+------------------------+------------------------+-------------------+------------------+
|            user_id|tweet_real_news_count|tweet_fake_news_count|tweet_count|retweet_real_news_count|retweet_fake_news_count|retweet_count|tweet_retweet_fake_count|tweet_retweet_real_count|tweet_retweet_count|   percentage_fake|
+-------------------+---------------------+---------------------+-----------+-----------------------+-----------------------+-------------+------------------------+------------------------+-------------------+------------------+
|1000011520099737600|                    1|                    0|          1|                      0|                      0|            0|                       0|                       1|                  1|               0.0|
|1000015310651645953|                    1|                    0|          1|       

In [115]:
#tweet_retweet_counts_df.toPandas().to_csv('tweet_retweet_counts.csv', index = False)

In [117]:
df_test = pd.read_csv('tweet_retweet_counts.csv')
df_test

,user_id,tweet_real_news_count,tweet_fake_news_count,tweet_count,retweet_real_news_count,retweet_fake_news_count,retweet_count,tweet_retweet_fake_count,tweet_retweet_real_count,tweet_retweet_count,percentage_fake
0,1000011520099737600,1,0,1,0,0,0,0,1,1,0.0
1,1000015310651645953,1,0,1,0,0,0,0,1,1,0.0
2,1000016148920913920,1,0,1,0,0,0,0,1,1,0.0
3,1000017210381819904,1,0,1,0,0,0,0,1,1,0.0
4,100002112,2,0,2,0,0,0,0,2,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...
364759,999970415748435969,4,0,4,0,0,0,0,4,4,0.0
364760,999973035540209664,8,0,8,0,0,0,0,8,8,0.0
364761,999973630195961856,1,0,1,0,0,0,0,1,1,0.0
364762,999975256981942273,0,1,1,0,0,0,1,0,1,1.0
